# Support Data Insight Analysis 📈

## Introduction 
This notebook provides a `comprehensive analysis of support data using CrewAI`. The analysis includes setting up the environment, loading task and agent configurations, utilizing file read tools, creating agents and tasks, and generating a final report. We aim to `derive actionable insights for improving the support process`.

In [ ]:
# Install required packages
%pip install "crewai[tools]"
%pip install "crewai[all]"

### Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

# Import necessary libraries
import os
import yaml
from crewai import Agent, Task, Crew

### Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

### Using FileReadTool

 > _Note_: Load the environment variables from a .env file to access the API keys. For more info see `.env.example`, [SERPER](https://serper.dev/?utm_term=google%20search%20api&gad_source=1&gad_campaignid=18303173259&gclid=EAIaIQobChMI4NXAsdOgjwMV6tJEBx3znCKdEAAYASAAEgL6kPD_BwE), and [OPENAI](https://platform.openai.com/api-keys). 

In [ ]:
import os
from crewai import Agent, Task, Crew

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
)

from helper import get_openai_api_key, get_serper_api_key
# Set up API keys
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["SERPER_API_KEY"] = get_serper_api_key()

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./support_tickets_data.csv')
file_tool = FileReadTool()  #this is what you passed to agent's tool
search_tool = SerperDevTool()

### Creating Agents, Tasks and Crew

In [ ]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  #tools=[csv_tool]
  #tools=[file_tool]
  tools=[docs_tool, file_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  #tools=[csv_tool]
  #tools=[file_tool]
  tools=[docs_tool, file_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=False
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)

### Executing Tasks

In [5]:
# Execute tasks
support_report_crew.kickoff()

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-


# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,custom

CrewOutput(raw='**Final Report on Support System Performance**\n\n---\n\n### 1. Issue Classification Results\n**Overview of the Types of Issues Reported, Their Frequency, and Priority Levels**\n\n| Issue Type         | Frequency | Priority Level          | Suggested Actions                                              |\n|--------------------|-----------|-------------------------|---------------------------------------------------------------|\n| API Issue          | 8         | High & Critical         | Improve API documentation, offer timely updates.             |\n| Login Issue        | 8         | High, Medium, Critical  | Streamline login processes, enhance authentication steps.     |\n| Data Import        | 9         | High, Medium            | Optimize data handling processes, increase server capacity.   |\n| Billing Issue      | 11        | High, Medium, Critical  | Ensure timely billing notifications, offer clearer payment details. |\n| Feature Request    | 6         | Medium,

### Testing our Crew
Now, let's ensure that the defined tasks and agents work correctly and produce the expected output. To do this the support_report_crew.test(n_iterations=1, openai_model_name="gpt-4o") snippet will run a test of the support_report_crew with a single iteration, using the "gpt-4o" model from OpenAI to evaluate the functionality and performance of the crew's tasks and agents. 

In [ ]:
# Runs a test for support_report_crew with one iteration using the GPT-4o model.
support_report_crew.test(n_iterations=1, openai_model_name="gpt-4o")

2025-02-26 20:51:16,668 - 16684 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: List files in directory
Tool Arguments: {}
Tool Description: A tool that can be used to list ./support_tickets_data.csv's content.
Tool Name: Read a file's content
Tool Arguments: 

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  8.0  │    8.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.5  │    9.5     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.5  │    8.5     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.50  │    8.5     │                     │  │  ┃
┃ Execution Time (s) │  53   │     53     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

**Explanation output** The output displays a summary of the performance scores for various tasks executed by different agents within a crew. Each task is rated on a scale of 1-10, with higher scores being better. The table includes the following columns:

- Tasks/Crew/Agents: The name of the task or crew.  
- Run 1: The score achieved in the first run.  
- Avg. Total: The average score for the task or crew.  
- Agents: The agent responsible for the task.  
- Additionally, the output includes the overall score for the crew and the total execution time in seconds.

### Training the crew and agents
Now,let's train the crew and agents. This process helps in refining the agents and tasks within the crew for better performance. To ensure this the support_report_crew will be trained  with one iteration and saves the training data to 'training.pkl'.

In [ ]:
# Trains the support_report_crew with one iteration and saves the training data to 'training.pkl'.
support_report_crew.train(n_iterations=1, filename='training.pkl')

2025-02-26 20:52:22,880 - 16684 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: List files in directory
Tool Arguments: {}
Tool Description: A tool that can be used to list ./support_tickets_data.csv's content.
Tool Name: Read a file's content
Tool Arguments: 

The overview of the four inputs that have been enclosed interactivly during the training of the crew and their task is the following:   
**Input1** I want you to do better meaniful to improve out the support quality  
**Input2** Make sure to also enclose an extra table with a great comparison we can actually use  
**Input3** This is great!  
**Input4** Make sure to always include the chart visuals close to the information related to them, like the tables, and make sure the reports is completed and is not missing any necessary information, including great suggestions.   

In [8]:
with open("training.pkl", "rb") as f:
    raw_data = f.read()  # Read raw bytes
    print("Raw File Content:", raw_data)

Raw File Content: b'\x80\x04\x95\x0f\r\x00\x00\x00\x00\x00\x00}\x94(\x8c\x12Suggestion Engine\n\x94}\x94(\x8c\x0bsuggestions\x94]\x94(\x8c\x8cFocus on gathering detailed customer feedback to improve service quality and identify the core issues causing delays in support resolutions.\x94\x8c\x82Implement a regular follow-up system to ensure customer satisfaction after resolving issues, allowing for personalized engagement.\x94\x8c\x8dEnhance the training materials for support agents by integrating customer insights and feedback, aiming for a more responsive support system.\x94\x8c\x85Create clearer communication protocols to keep customers informed during the issue resolution process, building trust and confidence.\x94\x8c\x89Utilize customer feedback for systemic improvements and to address common issues proactively, ensuring quicker resolutions in the future.\x94e\x8c\x07quality\x94G@ \x00\x00\x00\x00\x00\x00\x8c\rfinal_summary\x94XW\x02\x00\x001. Establish a system for collecting and 

In [ ]:
import pickle
import json

# Load the training_data.pkl file
with open("training.pkl", "rb") as f:
    training_data = pickle.load(f)  # Unpickle the data

# Check if the file contains valid data
if training_data:
    print("Successfully loaded training data!")

    # Pretty-print if it's a dictionary
    if isinstance(training_data, dict):
        print(json.dumps(training_data, indent=4, ensure_ascii=False))

    # If it's a list of dictionaries, format each entry
    elif isinstance(training_data, list):
        for i, entry in enumerate(training_data, 1):
            print(f"\n Entry {i}:\n{json.dumps(entry, indent=4, ensure_ascii=False)}")

    else:
        print("Data Structure:", type(training_data))
        print(training_data)  # Print directly if not JSON-compatible
else:
    print(" The training_data.pkl file is empty or contains None.")


✅ Successfully loaded training data!
{
    "Suggestion Engine\n": {
        "suggestions": [
            "Focus on gathering detailed customer feedback to improve service quality and identify the core issues causing delays in support resolutions.",
            "Implement a regular follow-up system to ensure customer satisfaction after resolving issues, allowing for personalized engagement.",
            "Enhance the training materials for support agents by integrating customer insights and feedback, aiming for a more responsive support system.",
            "Create clearer communication protocols to keep customers informed during the issue resolution process, building trust and confidence.",
            "Utilize customer feedback for systemic improvements and to address common issues proactively, ensuring quicker resolutions in the future."
        ],
        "quality": 8.0,
        "final_summary": "1. Establish a system for collecting and analyzing customer feedback frequently to ide

### Comparing new test results

In [ ]:
# Re-run a test for support_report_crew with one iteration using the GPT-4o model.
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-02-26 20:55:42,328 - 16684 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: List files in directory
Tool Arguments: {}
Tool Description: A tool that can be used to list ./support_tickets_data.csv's content.
Tool Name: Read a file's content
Tool Arguments: 

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  8.0  │    8.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.5  │    9.5     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.5  │    8.5     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.56  │    8.6     │                     │  │  ┃
┃ Execution Time (s) │  87   │     87     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

### Kicking off Crew
The support_report_crew.kickoff() method will start the execution of all tasks and agents in the support_report_crew object, generating reports and insights from support data.

In [11]:
result = support_report_crew.kickoff()

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to gather the necessary data from the support tickets to analyze and generate actionable suggestions. I will start by listing the files in the directory to find the relevant support ticket data.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-


You ONLY have access to the following tools, and should NEVER make up tools that

### Result

In [ ]:
# Display the resutls of the report
from IPython.display import display, Markdown
display(Markdown(result.raw))

### Final Report on Support Performance

#### Issue Classification Results
The following table summarizes the types of issues reported, their frequency, and priority levels:

| Issue Type           | Frequency | High Priority | Medium Priority | Low Priority | Critical Priority |
|----------------------|-----------|---------------|-----------------|--------------|-------------------|
| API Issue            | 10        | 5             | 5               | 0            | 4                 |
| Login Issue          | 9         | 4             | 2               | 0            | 1                 |
| Report Generation     | 6         | 2             | 1               | 1            | 1                 |
| Data Import          | 10        | 4             | 3               | 2            | 1                 |
| Feature Request      | 7         | 3             | 3               | 1            | 0                 |
| Billing Issue        | 8         | 5             | 2               | 1            | 1                 |
| UI Bug               | 5         | 1             | 3               | 1            | 0                 |

- **Key Trends**: The predominant issues reported relate to the API and Data Import, which could require enhanced focus for resolution strategies. Critical and High priority levels indicate urgency in addressing these matters.

#### Agent Performance
Performance of support agents is assessed using resolution time and customer satisfaction metrics:

| Agent ID | Average Resolution Time (minutes) | Average Satisfaction Rating |
|----------|-----------------------------------|-----------------------------|
| A001     | 958                               | 3.0                         |
| A002     | 792                               | 3.67                        |
| A003     | 817                               | 2.86                        |
| A004     | 785                               | 3.5                         |
| A005     | 733                               | 4.0                         |

- **Key Observations**: Agent A005 performed the best with the highest satisfaction rating and the lowest average resolution time. Agent A003 shows the most room for improvement in both categories.

#### Customer Satisfaction Over Time
The following table provides insights into customer satisfaction trends over recent months:

| Month  | Average Rating | Total Tickets |
|--------|----------------|---------------|
| Jan    | 3.8            | 8             |
| Feb    | 3.4            | 12            |
| Mar    | 3.67           | 10            |
| Apr    | 3.5            | 11            |
| May    | 3.33           | 11            |
| Jun    | 3.2            | 10            |
| Jul    | 3.0            | 1             |

- **Trend Analysis**: Average ratings have shown a downward trend, notably dropping in July. Understanding the reason for this decline will be essential for future customer satisfaction improvements.

#### Suggested Actions
Based on the review of support tickets, the following actions are recommended:

1. **For API Issues**: Investigate and address the root causes of delays; implement regular updates on resolutions to keep customers informed.
2. **For Login Issues**: Enhance the login process to minimize system errors and improve user experience.
3. **For Report Generation Problems**: Provide faster resolutions and introduce a fast-track process for urgent report-related requests.
4. **For Data Import Delays**: Streamline the data import framework and conduct training to help agents resolve these issues more swiftly.
5. **For Billing Issues**: Ensure that agents are trained in financial processes to avoid confusion and expedite resolutions.
6. **For UI Bugs**: Review the UI elements causing issues and prioritize fixes to prevent recurring tickets.
7. **Agent Performance**: Offer tailored training for agents performing below average to enhance their problem-solving skills and customer interaction techniques.

#### Charts and Visuals
To enhance understanding, charts showing issue distribution, resolution times, and customer satisfaction trends should be incorporated into stakeholder presentations.

The integrated analysis of these insights will empower stakeholders to make informed decisions, enhancing customer support performance and satisfaction.
```

## Conclusion
In this notebook, we successfully `set up CrewAI to analyze support data, created agents and tasks, and generated a comprehensive final report`. The insights derived from this analysis can help improve the support process and enhance customer satisfaction.